In [1]:
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing as hp


In [3]:
"""
#Start Refinitiv Session
rd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")
"""

'\n#Start Refinitiv Session\nrd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")\n'

In [5]:
"""
#Define instruments and Date range
START_DATE = "2006-01-01"
END_DATE = "2025-06-30"

# Replace these placeholders with the actual Refinitiv RICs 
SPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)
DOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index
WTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)
"""

'\n#Define instruments and Date range\nSTART_DATE = "2006-01-01"\nEND_DATE = "2025-06-30"\n\n# Replace these placeholders with the actual Refinitiv RICs \nSPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)\nDOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index\nWTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)\n'

In [7]:
#import refinitiv.data as rd
#print("Refinitiv Data Library version:", rd.__version__)

#print(dir(hp))

"""
from refinitiv.data.content import pricing

# request latest snapshot for Apple
snap = pricing.Definition(universe="AAPL.O").get_data()
print(snap)
"""

"""
#Obtain required data
# --- 1. S&P 500 OHLCV ---
df_spx_raw = hp.summaries.Definition(
    universe=SPX_PROXY_RIC,
    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# --- 2. Dollar Index & WTI Oil (Closing Prices) ---
df_macro_raw = hp.summaries.Definition(
    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],
    fields=["CLOSE"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# Output checks
print("S&P 500 Raw DataFrame (df_spx_raw):")
print(df_spx_raw.head())

print("\nMacro Raw DataFrame (df_macro_raw):")
print(df_macro_raw.head())
"""

'\n#Obtain required data\n# --- 1. S&P 500 OHLCV ---\ndf_spx_raw = hp.summaries.Definition(\n    universe=SPX_PROXY_RIC,\n    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# --- 2. Dollar Index & WTI Oil (Closing Prices) ---\ndf_macro_raw = hp.summaries.Definition(\n    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],\n    fields=["CLOSE"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# Output checks\nprint("S&P 500 Raw DataFrame (df_spx_raw):")\nprint(df_spx_raw.head())\n\nprint("\nMacro Raw DataFrame (df_macro_raw):")\nprint(df_macro_raw.head())\n'

In [ ]:
#Stationarity checks
#SPY log returns
adf_result = adfuller(df['SPY_log_ret'])
print(f"ADF Statistic: {adf_result[0]:.3f}, p-value: {adf_result[1]:.5f}")
if adf_result[1] < 0.05:
    print("✅ SPY_log_ret is stationary.")
else:
    print("⚠️ SPY_log_ret may not be stationary; differencing might be needed.")


#SPY log returns
adf_result_1 = adfuller(df['DXY_ret'])
print(f"ADF Statistic: {adf_result[0]:.3f}, p-value: {adf_result[1]:.5f}")
if adf_result[1] < 0.05:
    print("✅ DXY_ret is stationary.")
else:
    print("⚠️ DXY_ret may not be stationary; differencing might be needed.")


#SPY log returns
adf_result_2 = adfuller(df['WTI_ret'])
print(f"ADF Statistic: {adf_result[0]:.3f}, p-value: {adf_result[1]:.5f}")
if adf_result[1] < 0.05:
    print("✅ WTI_ret is stationary.")
else:
    print("⚠️ WTI_ret may not be stationary; differencing might be needed.")

In [ ]:
#Scale numerical features
features_to_scale = ['SPY_log_ret', 'SPY_p_change', 'DXY_ret', 'WTI_ret']
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[features_to_scale] = scaler.fit_transform(df_scaled[features_to_scale])

#Check output
print("\nScaled feature sample:")
print(df_scaled[features_to_scale].head())

# Adaptive Wavelet Transform (AWT) Denoising & Decomposition

In [ ]:
#Import required libraries 
import pywt
import matplotlib.pyplot as plt

#1. Extract target signal
signal = df_scaled['SPY_log_ret'].values  # denoised target variable

#2. Choose wavelet parameters
wavelet = 'db4'         # Daubechies 4 is widely used in financial time series
level = 2               # decomposition level (2–4 typical for daily data)

#3. Perform discrete wavelet decomposition
coeffs = pywt.wavedec(signal, wavelet, level=level)

# where coeffs[0] = Approximation (trend), and
# coeffs[1:] = Detail coefficients (short-term fluctuations)

#4. Reconstruct the approximation and detail signals
approximation = pywt.waverec([coeffs[0]] + [None]*level, wavelet)
detail = signal - approximation[:len(signal)]  # ensure same length

# Clip both to the same index as df_scaled
approximation = approximation[:len(df_scaled)]
detail = detail[:len(df_scaled)]

#5. Add components to your DataFrame
df_scaled['SPY_s'] = approximation
df_scaled['SPY_d'] = detail

#6. Visual check of output
plt.figure(figsize=(12,6))
plt.plot(df_scaled.index, df_scaled['SPY_log_ret'], label='Original', alpha=0.6)
plt.plot(df_scaled.index, df_scaled['SPY_s'], label='Approximation (Trend)', color='orange')
plt.plot(df_scaled.index, df_scaled['SPY_d'], label='Detail (Fluctuation)', color='green', alpha=0.5)
plt.legend()
plt.title("AWT Decomposition of SPY_log_ret using Daubechies-4")
plt.show()

#7. Save the decomposed dataset
df_scaled.to_csv("03_awt_decomposed_data.csv")
print("\n✅ AWT decomposition complete. Data saved as '03_awt_decomposed_data.csv'")

# Feature Engineering: 21 Financial Technical Indicators

In [ ]:
# ================================================================
# DATA PREPARATION & MERGE (for yfinance-style Data)
# ================================================================

#Import required libraries
import re
import ta  # technical analysis library

# --- 1️⃣ Flatten MultiIndex columns if any ---
def flatten_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(c) for c in col if c]) for col in df.columns]
    df.columns = [str(c).strip() for c in df.columns]
    return df

df_spx_full = flatten_columns(df_spx_raw)
df_scaled = flatten_columns(df_scaled)

# --- 2️⃣ Identify & rename likely OHLCV columns ---
ohlcv_candidates = [c for c in df_spx_full.columns if any(x in c.lower() for x in ['open', 'high', 'low', 'close', 'volume'])]
rename_map = {}
for col in ohlcv_candidates:
    lc = col.lower()
    if 'open' in lc: rename_map[col] = 'SPY_Open'
    elif 'high' in lc: rename_map[col] = 'SPY_High'
    elif 'low' in lc: rename_map[col] = 'SPY_Low'
    elif 'close' in lc and 'adj' not in lc: rename_map[col] = 'SPY_Close'
    elif 'adj' in lc and 'close' in lc: rename_map[col] = 'SPY_Adj_Close'
    elif 'volume' in lc: rename_map[col] = 'SPY_Volume'

df_spx_full = df_spx_full.rename(columns=rename_map)

# --- 3️⃣ Drop duplicates and merge safely ---
df_scaled = df_scaled.drop(columns=[c for c in ['SPY_Close'] if c in df_scaled.columns], errors='ignore')
df_feat = df_scaled.join(
    df_spx_full[[c for c in ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume'] if c in df_spx_full.columns]],
    how='inner',
    rsuffix='_orig'
)

# --- 4️⃣ Clean up any whitespace or weird naming ---
df_feat.columns = df_feat.columns.map(lambda x: re.sub(r'\s+', '', str(x)))

# --- 5️⃣ Confirm required columns exist ---
required_cols = ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume']
missing = [c for c in required_cols if c not in df_feat.columns]
if missing:
    raise KeyError(f"Missing SPY OHLCV columns: {missing}")
else:
    print("✅ All SPY OHLCV columns present.")

print(f"Final df_feat shape before feature generation: {df_feat.shape}")
print(df_feat.columns.tolist())


In [ ]:
# ================================================================
# FEATURE ENGINEERING: 21 TECHNICAL INDICATORS (CLEAN VERSION)
# ================================================================

import pandas as pd
import numpy as np
import ta  # technical analysis library

# ------------------------------------------------
# 0️⃣ Ensure clean, unique index
# ------------------------------------------------
df_feat = df_feat.copy()
df_feat = df_feat[~df_feat.index.duplicated(keep='first')]
df_feat = df_feat.sort_index()
print(f"✅ Index cleaned. Unique rows: {len(df_feat)}")

# ------------------------------------------------
# 1️⃣ Confirm required columns
# ------------------------------------------------
required_cols = ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume', 'SPY_s']
for c in required_cols:
    if c not in df_feat.columns:
        raise KeyError(f"Missing required column: {c}")

print("✅ All required columns confirmed present.\n")

# ------------------------------------------------
# 2️⃣ Moving Averages (Trend)
# ------------------------------------------------
price_col = 'SPY_s'
vol_col = 'SPY_Volume'

df_feat['EMA20'] = ta.trend.EMAIndicator(df_feat[price_col], window=20).ema_indicator()
df_feat['MA5'] = ta.trend.SMAIndicator(df_feat[price_col], window=5).sma_indicator()
df_feat['MA10'] = ta.trend.SMAIndicator(df_feat[price_col], window=10).sma_indicator()
df_feat['V_MA5'] = ta.trend.SMAIndicator(df_feat[vol_col], window=5).sma_indicator()
df_feat['V_MA10'] = ta.trend.SMAIndicator(df_feat[vol_col], window=10).sma_indicator()

# ------------------------------------------------
# 3️⃣ Momentum Indicators
# ------------------------------------------------
df_feat['MACD'] = ta.trend.MACD(df_feat[price_col]).macd()
df_feat['ROC'] = ta.momentum.ROCIndicator(df_feat[price_col], window=10).roc()
df_feat['MTM6'] = df_feat[price_col] / df_feat[price_col].shift(126) - 1
df_feat['MTM12'] = df_feat[price_col] / df_feat[price_col].shift(252) - 1

stoch = ta.momentum.StochasticOscillator(
    high=df_feat['SPY_High'],
    low=df_feat['SPY_Low'],
    close=df_feat[price_col],
    window=14,
    smooth_window=3
)
df_feat['SMI'] = stoch.stoch_signal()

# ------------------------------------------------
# 4️⃣ Volatility & Pattern Indicators
# ------------------------------------------------
df_feat['ATR'] = ta.volatility.AverageTrueRange(
    high=df_feat['SPY_High'],
    low=df_feat['SPY_Low'],
    close=df_feat[price_col],
    window=14
).average_true_range()

boll = ta.volatility.BollingerBands(df_feat[price_col], window=20, window_dev=2)
df_feat['BOLL_upper'] = boll.bollinger_hband()
df_feat['BOLL_lower'] = boll.bollinger_lband()
df_feat['BOLL_bandwidth'] = df_feat['BOLL_upper'] - df_feat['BOLL_lower']

df_feat['CCI'] = ta.trend.CCIIndicator(
    high=df_feat['SPY_High'],
    low=df_feat['SPY_Low'],
    close=df_feat[price_col],
    window=20
).cci()

# ------------------------------------------------
# 5️⃣ Volume-based Indicator (WVAD)
# ------------------------------------------------
# Ensure alignment and valid numeric dtype
for c in ['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume']:
    df_feat[c] = pd.to_numeric(df_feat[c], errors='coerce')

df_feat['WVAD'] = (
    (df_feat['SPY_Close'] - df_feat['SPY_Open']) /
    (df_feat['SPY_High'] - df_feat['SPY_Low'] + 1e-10)
) * df_feat['SPY_Volume']

# ------------------------------------------------
# 6️⃣ Final Cleanup
# ------------------------------------------------
df_feat = df_feat.replace([np.inf, -np.inf], np.nan).dropna()
print(f"✅ Technical indicators successfully generated! Final shape: {df_feat.shape}")

# ------------------------------------------------
# 7️⃣ Save to file
# ------------------------------------------------
df_feat.to_csv("04_feature_engineered_data.csv")
print("💾 Saved as '04_feature_engineered_data.csv'")


In [ ]:
# 3️⃣ OPTIONAL VISUAL CHECKS
# ================================================================

# --- Plot MACD ---
plt.figure(figsize=(12, 5))
plt.plot(df_feat.index, df_feat[price_col], label='SPY_s (Smoothed Price)', color='black', alpha=0.7)
plt.plot(df_feat.index, df_feat['EMA20'], label='EMA20', linestyle='--')
plt.title("Price with EMA20")
plt.legend()
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(df_feat.index, df_feat['MACD'], label='MACD', color='blue')
plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.title("MACD Indicator")
plt.legend()
plt.show()

# --- Plot Bollinger Bands ---
plt.figure(figsize=(12, 5))
plt.plot(df_feat.index, df_feat[price_col], label='SPY_s (Smoothed Price)', color='black')
plt.plot(df_feat.index, df_feat['BOLL_upper'], label='Bollinger Upper', linestyle='--', color='red')
plt.plot(df_feat.index, df_feat['BOLL_lower'], label='Bollinger Lower', linestyle='--', color='green')
plt.fill_between(df_feat.index, df_feat['BOLL_lower'], df_feat['BOLL_upper'], color='gray', alpha=0.1)
plt.title("Bollinger Bands")
plt.legend()
plt.show()

# --- Plot Volume & WVAD ---
plt.figure(figsize=(12, 4))
plt.plot(df_feat.index, df_feat['SPY_Volume'], label='SPY Volume', color='orange', alpha=0.6)
plt.plot(df_feat.index, df_feat['WVAD'], label='WVAD', color='purple', alpha=0.8)
plt.title("Volume and WVAD")
plt.legend()
plt.show()

In [ ]:
# ================================================================
# TWO-PANEL VERIFICATION PLOT: Price + Bollinger (top), MACD (bottom)
# ================================================================
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# --- Safety: ensure needed columns are present ---
_need = ['SPY_s', 'BOLL_upper', 'BOLL_lower', 'MACD']
_missing = [c for c in _need if c not in df_feat.columns]
if _missing:
    raise KeyError(f"Missing columns for plot: {_missing}")

# Optional: focus window (e.g., last 2 years). Comment out to use full history.
# dfp = df_feat.last('730D')  # last ~730 days
dfp = df_feat.copy()

fig = plt.figure(figsize=(14, 8))
gs = fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1.4], hspace=0.08)

# -------------------------
# Top panel: Price + Bands
# -------------------------
ax_price = fig.add_subplot(gs[0, 0])
ax_price.plot(dfp.index, dfp['SPY_s'], label='SPY_s (Smoothed Price)')
ax_price.plot(dfp.index, dfp['BOLL_upper'], label='Bollinger Upper', linestyle='--')
ax_price.plot(dfp.index, dfp['BOLL_lower'], label='Bollinger Lower', linestyle='--')
ax_price.fill_between(dfp.index, dfp['BOLL_lower'], dfp['BOLL_upper'], alpha=0.10)
ax_price.set_title("SPY_s with Bollinger Bands")
ax_price.legend(loc='upper left')
ax_price.grid(True, alpha=0.3)

# -------------------------
# Bottom panel: MACD
# -------------------------
ax_macd = fig.add_subplot(gs[1, 0], sharex=ax_price)
ax_macd.plot(dfp.index, dfp['MACD'], label='MACD')
ax_macd.axhline(0.0, linestyle='--', linewidth=1)
ax_macd.set_title("MACD")
ax_macd.legend(loc='upper left')
ax_macd.grid(True, alpha=0.3)

# Tidy x-axis formatting
date_fmt = DateFormatter("%Y-%m")
ax_macd.xaxis.set_major_formatter(date_fmt)
plt.setp(ax_price.get_xticklabels(), visible=False)

plt.show()